# Herein lies the code to plot every figure on my Summer 2025 SURA report #

# Imports #

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os
import cmocean as cm
import waypoint_distance as wd
import pandas as pd
from pathlib import Path
from datetime import datetime
from matplotlib.dates import DateFormatter
import gsw
import matplotlib.dates as mdates
%matplotlib widget

cube = xr.open_dataset(os.path.expanduser('~/Desktop/Summer 2025 Python/calvert_cube.nc'))
topo = xr.open_dataset(os.path.expanduser('~/Desktop/Summer 2025 Python/british_columbia_3_msl_2013.nc'))
ds = xr.open_dataset('~/Desktop/Summer 2025 Python/Hakai_calvert.nc')

# Mapping #

In [ ]:
def plot_temp_time_series_subplots(
    cube,
    along_values,
    target_years=[2019, 2020, 2021, 2022, 2023, 2024, 2025],
    ylims=[(400,0), (130,0), (200,0)],
    start_month=None
):
    import matplotlib.pyplot as plt
    from matplotlib.dates import DateFormatter
    import pandas as pd
    import numpy as np
    from datetime import datetime
    import cmocean as cm

    # Calculate height ratios based on ylims
    max_depths = [yl[0] for yl in ylims]
    max_depth = max(max_depths)
    height_ratios = [d / max_depth for d in max_depths]

    n = len(along_values)
    fig, axes = plt.subplots(
        nrows=n, figsize=(1.3 * 16, 1.2 * 14),
        constrained_layout=True,
        gridspec_kw={'height_ratios': height_ratios}
    )
    fig.suptitle("Temperature Time Series Along Transect", fontsize=40)
    if n == 1:
        axes = [axes]

    # Collect all valid times to compute global xlim
    all_times = []
    time_lookup = {}
    for along_value in along_values:
        cube_sel = cube.sel(along=along_value)
        times_this = [datetime.strptime(t[:8], "%Y%m%d") for t in cube_sel.transect.values]
        valid_times = [
            t for t in times_this
            if t.year in target_years and (start_month is None or t >= datetime(target_years[0], start_month, 1))
        ]
        all_times.extend(valid_times)
        time_lookup[along_value] = valid_times

    if not all_times:
        print("No valid time data found.")
        return

    global_xlim = (min(all_times), max(all_times))

    for ax, along_value, ylim in zip(axes, along_values, ylims):
        cube_sel = cube.sel(along=along_value)

        temp = cube_sel['temperature'].values
        density = cube_sel['potential_density'].values - 1000
        depth = cube_sel['depth'].values
        times = [datetime.strptime(t[:8], "%Y%m%d") for t in cube_sel.transect.values]

        filtered = [
            (t, te, de)
            for t, te, de in zip(times, temp, density)
            if (
                t.year in target_years
                and (start_month is None or t >= datetime(target_years[0], start_month, 1))
                and not np.all(np.isnan(te))
            )
        ]

        if not filtered:
            print(f"No data for along={along_value}")
            continue

        times_data, temp_data, density_data = zip(*sorted(filtered))
        this_times = sorted(set(times_data))
        temp_grid = np.full((temp_data[0].shape[0], len(this_times)), np.nan)
        density_grid = np.full_like(temp_grid, np.nan)

        for i, t in enumerate(times_data):
            time_idx = this_times.index(t)
            temp_grid[:, time_idx] = temp_data[i]
            density_grid[:, time_idx] = density_data[i]

        pc = ax.pcolormesh(this_times, depth, temp_grid,
                           cmap=cm.cm.thermal, shading='nearest',
                           vmin=5.3, vmax=10)

        for levels, color, lw in [
                (np.linspace(24, 27, 7), 'black', .51),
                            ([25.6], 'white', 0.5),
                            ([25.7], 'lime', 0.5),
                            ([25.8], 'red', 0.5),
                            ([25.9], 'blue', 0.5),
                            ([26.0], 'black', 0.51),
                            ([26.1], 'purple', 0.5),
                            ([26.2], 'salmon', 0.5),
                            ([26.3], 'yellow', 0.5),
                            ([26.4], 'cyan', 0.5)]:
            cf_iso = ax.contour(this_times, depth, density_grid, levels=levels, colors=color, linewidths=lw)
            if lw != 0.3:
                ax.clabel(cf_iso, fmt='%1.2f')

        ax.text(0.01, 0.95,
                f"{along_value / 1000:.1f} km",
                transform=ax.transAxes,
                fontsize=30,
                fontweight="bold",
                va="top", ha="left")

        ax.invert_yaxis()
        ax.set_ylim(ylim)
        ax.set_xlim(global_xlim)
        ax.xaxis.set_major_formatter(DateFormatter('%b %d\n%Y'))

        if ax is not axes[-1]:
            ax.set_xticklabels([])

        if ax is axes[0]:
            ax_top = ax.secondary_xaxis('top')
            ax_top.set_xticks(pd.date_range(global_xlim[0], global_xlim[1], periods=len(this_times)))
            ax_top.set_xticklabels(
                [pd.Timestamp(t).strftime('%b %-d') for t in this_times],
                rotation=90, fontsize=16
            )
            ax_top.tick_params(axis='x', direction='out', length=3, width=0.5, pad=2)

    plt.rcParams.update({
        "xtick.labelsize": 30,
        "ytick.labelsize": 30,
        "axes.titlesize": 30,
        "axes.labelsize": 30,
        "figure.titlesize": 40
    })

    fig.colorbar(pc, ax=axes, label="Temperature (°C)", orientation='vertical')
    fig.supylabel("Depth (m)", fontsize=30)
    # fig.savefig(os.path.expanduser("~/Desktop/along_time_series.pdf"), format='pdf', bbox_inches='tight', dpi = 600)

plot_temp_time_series_subplots(
    cube,
    along_values=[65000, 23500, 6000],
    ylims=[(200,0), (130,0), (400,0)],
    target_years=[2024,2025], start_month = 1)

# Plot temperature sections #

In [ ]:
def plot_temperature_sections(cube, topo, target_years, ncols=4, xlim=(77, 0), vmin=5.3, vmax=10):
    """
    Plot a grid of temperature sections from a transect cube, filtered by year.

    Parameters:
    - cube: xarray.Dataset with dimensions (transect, depth, along)
    - topo: bathymetry dataset
    - target_years: list of years to include
    - ncols: number of columns in the grid
    - xlim: x-axis limits for each subplot
    """

    plt.rcParams.update({
        'font.size': 12,
        'axes.titlesize': 20,
        'axes.labelsize': 20,
        'xtick.labelsize': 15,
        'ytick.labelsize': 15,
        'legend.fontsize':  20,
        'figure.titlesize': 20})
    
    # ─── Filter transects by year ─────────────────────────────
    transects = cube.transect.values
    selected = [str(t) for t in transects if int(str(t)[:4]) in target_years]
    cube_sel = cube.sel(transect=selected)
    times = [datetime.strptime(t[:8], "%Y%m%d") for t in cube.transect.values]

    n = len(selected)
    nrows = int(np.ceil(n / ncols))

    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 1.5 * 6.4, nrows * 1 * 4.8), sharex=True, sharey=True)
    axes = axes.flatten()

    for i, tran_name in enumerate(selected):
        ds = cube_sel.sel(transect=tran_name)

        ax = axes[i]
        temp = ds['temperature'].values
        pdens = ds['potential_density'].values - 1000
        depth = ds['depth'].values
        along = ds['along'].values
        lon = ds['longitude'].values
        lat = ds['latitude'].values

        # Interpolate bathymetry
        interp_bathy = topo['Band1'].interp(
            lon=xr.DataArray(lon, dims='along'),
            lat=xr.DataArray(lat, dims='along'),
            method='nearest')
        ocean_floor = -interp_bathy.values

        # Fill below bathymetry
        ax.fill_between(along / 1000, ocean_floor, 500,
                        where=~np.isnan(ocean_floor), facecolor='grey', zorder=1)

        # Plot temperature
        pc = ax.pcolormesh(along / 1000, depth, temp,
                           shading='auto', cmap=cm.cm.thermal,
                           vmin=vmin, vmax=vmax, zorder=2, rasterized=True)

        # Bathymetry outline
        ax.plot(along / 1000, ocean_floor, color='black', linewidth=1.5)

        # Isopycnals
        for levels, color, lw in [
            (np.linspace(24, 27, 7), 'black', 0.5),
            ([25.6], 'white', 0.5),
            ([25.7], 'lime', 0.5),
            ([25.8], 'red', 0.5),
            ([25.9], 'blue', 0.5),
            ([26.0], 'black', 0.5),
            ([26.1], 'purple', 0.5),
            ([26.2], 'salmon', 0.5),
            ([26.3], 'yellow', 0.5),
            ([26.4], 'cyan', 0.5)]:
            iso = ax.contour(along / 1000, depth, pdens, levels=levels,
                             colors=color, linewidths=lw)
            ax.clabel(iso, fmt='%1.2f', fontsize=6)

        date_str = pd.to_datetime(tran_name[:8], format='%Y%m%d').strftime('%B %d')
        leg = 'Out' if 'out' in tran_name else 'Return'
        ax.set_title(f"{date_str} ({leg})", pad=2)
        
                # ─── Top Axis with Time Labels ──────────────────────
        if 'time_top' in ds:
            along_km = along / 1000
            time_top = ds['time_top'].values

            nticks = 8
            idx_ticks = np.linspace(0, len(along_km) - 1, nticks, dtype=int)
            tick_locs = along_km[idx_ticks]
            tick_times = time_top[idx_ticks]

            # Remove NaT or invalids
            valid_mask = ~pd.isnull(tick_times)
            tick_locs = tick_locs[valid_mask]
            tick_times = tick_times[valid_mask]
            tick_labels = [pd.to_datetime(t).strftime('%b %d %H:%M') for t in tick_times]

            ax_top = ax.secondary_xaxis('top')
            ax_top.set_xticks(tick_locs)
            ax_top.set_xticklabels(tick_labels, rotation=30, ha='center', fontsize=8)

            # Add minor ticks
            from matplotlib.ticker import AutoMinorLocator
            ax_top.xaxis.set_minor_locator(AutoMinorLocator(n=2))
            ax_top.tick_params(which='minor', length=3, width=0.8)

    ax.invert_yaxis()
    ax.invert_xaxis()
    ax.set_xlim(xlim)
    ax.set_ylim(440, 0)
    
    # Remove unused axes
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Adjust layout before placing colorbar and labels
    plt.tight_layout(rect=[0.03, 0.04, 0.85, 0.94])  # tighter left/right again

    cbar_ax = fig.add_axes([0.87, 0.25, 0.015, 0.5])  # move colorbar left again
    cbar = fig.colorbar(pc, cax=cbar_ax)
    cbar.ax.tick_params(labelsize=12)
    cbar.set_label("Temperature (°C)", fontsize=30)

    fig.suptitle(f"Temperature Sections ({', '.join(map(str, target_years))})", fontsize=40)
    # Adjusted subplot region: match rect used in tight_layout
    left, right = 0.03, 0.85  # These should match your tight_layout(..., rect=[...])
    x_center = (left + right) / 2

    fig.text(x_center, 0.01, 'Distance along transect (km)', ha='center', va='center', fontsize=30)
    fig.text(0.012, 0.5, 'Depth (m)', ha='center', va='center', rotation='vertical', fontsize=30)
    # fig.savefig(os.path.expanduser("~/Desktop/section_grid2.pdf"), format='pdf', bbox_inches='tight', dpi = 1000)

plot_temperature_sections(
    cube=cube,
    topo=topo,
    target_years=[2024, 2025],
    ncols=2,
    xlim=(77, 0),
    vmin=5.3,
    vmax=10)

# Temperature time series of different along values #

In [ ]:
def plot_temp_time_series_subplots(
    cube,
    along_values,
    target_years=[2019, 2020, 2021, 2022, 2023, 2024, 2025],
    ylims=[(400,0), (130,0), (200,0)],
    start_month=None
):
    import matplotlib.pyplot as plt
    from matplotlib.dates import DateFormatter
    import pandas as pd
    import numpy as np
    from datetime import datetime
    import cmocean as cm

    # Calculate height ratios based on ylims
    max_depths = [yl[0] for yl in ylims]
    max_depth = max(max_depths)
    height_ratios = [d / max_depth for d in max_depths]

    n = len(along_values)
    fig, axes = plt.subplots(
        nrows=n, figsize=(1.3 * 16, 1.2 * 14),
        constrained_layout=True,
        gridspec_kw={'height_ratios': height_ratios}
    )
    fig.suptitle("Temperature Time Series Along Transect", fontsize=40)
    if n == 1:
        axes = [axes]

    # Collect all valid times to compute global xlim
    all_times = []
    time_lookup = {}
    for along_value in along_values:
        cube_sel = cube.sel(along=along_value)
        times_this = [datetime.strptime(t[:8], "%Y%m%d") for t in cube_sel.transect.values]
        valid_times = [
            t for t in times_this
            if t.year in target_years and (start_month is None or t >= datetime(target_years[0], start_month, 1))
        ]
        all_times.extend(valid_times)
        time_lookup[along_value] = valid_times

    if not all_times:
        print("No valid time data found.")
        return

    global_xlim = (min(all_times), max(all_times))

    for ax, along_value, ylim in zip(axes, along_values, ylims):
        cube_sel = cube.sel(along=along_value)

        temp = cube_sel['temperature'].values
        density = cube_sel['potential_density'].values - 1000
        depth = cube_sel['depth'].values
        times = [datetime.strptime(t[:8], "%Y%m%d") for t in cube_sel.transect.values]

        filtered = [
            (t, te, de)
            for t, te, de in zip(times, temp, density)
            if (
                t.year in target_years
                and (start_month is None or t >= datetime(target_years[0], start_month, 1))
                and not np.all(np.isnan(te))
            )
        ]

        if not filtered:
            print(f"No data for along={along_value}")
            continue

        times_data, temp_data, density_data = zip(*sorted(filtered))
        this_times = sorted(set(times_data))
        temp_grid = np.full((temp_data[0].shape[0], len(this_times)), np.nan)
        density_grid = np.full_like(temp_grid, np.nan)

        for i, t in enumerate(times_data):
            time_idx = this_times.index(t)
            temp_grid[:, time_idx] = temp_data[i]
            density_grid[:, time_idx] = density_data[i]

        pc = ax.pcolormesh(this_times, depth, temp_grid,
                           cmap=cm.cm.thermal, shading='nearest',
                           vmin=5.3, vmax=10)

        for levels, color, lw in [
                (np.linspace(24, 27, 7), 'black', .51),
                            ([25.6], 'white', 0.5),
                            ([25.7], 'lime', 0.5),
                            ([25.8], 'red', 0.5),
                            ([25.9], 'blue', 0.5),
                            ([26.0], 'black', 0.51),
                            ([26.1], 'purple', 0.5),
                            ([26.2], 'salmon', 0.5),
                            ([26.3], 'yellow', 0.5),
                            ([26.4], 'cyan', 0.5)]:
            cf_iso = ax.contour(this_times, depth, density_grid, levels=levels, colors=color, linewidths=lw)
            if lw != 0.3:
                ax.clabel(cf_iso, fmt='%1.2f')

        ax.text(0.01, 0.95,
                f"{along_value / 1000:.1f} km",
                transform=ax.transAxes,
                fontsize=30,
                fontweight="bold",
                va="top", ha="left")

        ax.invert_yaxis()
        ax.set_ylim(ylim)
        ax.set_xlim(global_xlim)
        ax.xaxis.set_major_formatter(DateFormatter('%b %d\n%Y'))

        if ax is not axes[-1]:
            ax.set_xticklabels([])

        if ax is axes[0]:
            ax_top = ax.secondary_xaxis('top')
            ax_top.set_xticks(pd.date_range(global_xlim[0], global_xlim[1], periods=len(this_times)))
            ax_top.set_xticklabels(
                [pd.Timestamp(t).strftime('%b %-d') for t in this_times],
                rotation=90, fontsize=16
            )
            ax_top.tick_params(axis='x', direction='out', length=3, width=0.5, pad=2)

    plt.rcParams.update({
        "xtick.labelsize": 30,
        "ytick.labelsize": 30,
        "axes.titlesize": 30,
        "axes.labelsize": 30,
        "figure.titlesize": 40
    })

    fig.colorbar(pc, ax=axes, label="Temperature (°C)", orientation='vertical')
    fig.supylabel("Depth (m)", fontsize=30)
    # fig.savefig(os.path.expanduser("~/Desktop/along_time_series.pdf"), format='pdf', bbox_inches='tight', dpi = 600)

plot_temp_time_series_subplots(
    cube,
    along_values=[65000, 23500, 6000],
    ylims=[(200,0), (130,0), (400,0)],
    target_years=[2024,2025], start_month = 1)

# TS diagram for 0-30km with insets #

In [ ]:
def plot_ts_shelf_basin_by_transect(
    cube,
    region_range=(0, 30000),
    target_years=[2024, 2025],
    target_months=list(range(1, 13)),
    xlim=(30.5, 34),
    ylim=(5, 16),
    ncols=4,
    inset_months=[4, 5, 6, 7, 8, 9],
    inset_limits_by_date=None,
):
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import gsw
    from scipy.optimize import root_scalar
    from matplotlib.colors import Normalize
    import os

    plt.rcParams.update({
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
        "axes.titlesize": 14,
        "axes.labelsize": 12,
        "legend.fontsize": 10,
        "figure.titlesize": 16})

    cube = cube.load()
    subset = cube.sel(along=slice(*region_range))
    times = pd.to_datetime([str(t)[:8] for t in subset.transect.values])
    selected_idxs = [i for i, t in enumerate(times) if t.year in target_years and t.month in target_months]
    n_panels = len(selected_idxs)
    nrows = int(np.ceil(n_panels / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                            figsize=(4.5 * ncols, 4 * nrows), squeeze=False)

    fig.subplots_adjust(
        wspace=0.1,
        hspace=0.1)
    axes = axes.ravel()

    # Isopycnal grid
    S_grid, T_grid = np.meshgrid(np.linspace(28, 35, 300), np.linspace(5.3, 20, 300))
    sigma = gsw.sigma0(S_grid, T_grid)
    T_label = 6.0
    isopycnal_levels = [23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 25.6, 25.7, 25.8,
                        25.9, 26.0, 26.1, 26.2, 26.3, 26.4, 26.5]
    S_label = []
    for iso in isopycnal_levels:
        def func(S): return gsw.sigma0(S, T_label) - iso
        sol = root_scalar(func, bracket=[28, 35], method='brentq')
        S_label.append(sol.root if sol.converged else np.nan)
    manual_labels = {iso: (s, T_label) for iso, s in zip(isopycnal_levels, S_label)}

    cmap = plt.colormaps['jet']
    norm = Normalize(vmin=region_range[0], vmax=region_range[1])

    for i, idx in enumerate(selected_idxs):
        ax = axes[i]
        ax.set_facecolor('grey')

        for levels, color, lw in [
            (np.linspace(23, 27, 9), 'black', 0.5),
            ([25.6], 'white', 0.5), ([25.7], 'lime', 0.5),
            ([25.8], 'red', 0.5), ([25.9], 'blue', 0.5),
            ([26.0], 'black', 0.5), ([26.1], 'purple', 0.5),
            ([26.2], 'salmon', 0.5), ([26.3], 'yellow', 0.5),
            ([26.4], 'cyan', 0.5)]:
            cs = ax.contour(S_grid, T_grid, sigma, levels=levels,
                            colors=color, linewidths=lw, linestyles='--')
            for level in levels:
                if level in manual_labels:
                    ax.clabel(cs, fmt='%1.2f', fontsize=7, inline=False,
                              manual=[manual_labels[level]])

        ds = subset.isel(transect=idx)
        temp = ds['temperature']
        sali = ds['salinity']
        along_vals = ds['along'].values

        for a_idx, along in reversed(list(enumerate(along_vals))):
            T_col = temp.isel(along=a_idx).values
            S_col = sali.isel(along=a_idx).values
            mask = ~np.isnan(T_col) & ~np.isnan(S_col)
            if np.any(mask):
                ax.scatter(S_col[mask], T_col[mask], s=1.25, color=cmap(norm(along)), marker='o')

        ax.set_title(times[idx].strftime('%d %b %Y'))
        if i % ncols == 0:
            ax.set_ylabel("$\\theta$ (\u00b0C)")
        else:
            ax.set_yticklabels([])
        if i // ncols == nrows - 1:
            ax.set_xlabel("Salinity (psu)")
        else:
            ax.set_xticklabels([])
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)

        # ─── Add Inset ─────────────────────────────
        if times[idx].month in inset_months:
            inset = ax.inset_axes([0.5, 0.5, 0.48, 0.48])
            inset.set_facecolor('grey')
            inset_xlim_local = (33.3, 33.8)
            inset_ylim_local = (6.0, 7.1)

            date_str = times[idx].strftime('%Y-%m-%d')
            if inset_limits_by_date and date_str in inset_limits_by_date:
                inset_xlim_local = inset_limits_by_date[date_str].get('xlim', inset_xlim_local)
                inset_ylim_local = inset_limits_by_date[date_str].get('ylim', inset_ylim_local)

            for levels, color, lw in [
                (np.linspace(23, 27, 9), 'black', 0.5),
                ([25.6], 'white', 0.5), ([25.7], 'lime', 0.5),
                ([25.8], 'red', 0.5), ([25.9], 'blue', 0.5),
                ([26.0], 'black', 0.5), ([26.1], 'purple', 0.5),
                ([26.2], 'salmon', 0.5), ([26.3], 'yellow', 0.5),
                ([26.4], 'cyan', 0.5)]:
                cs = inset.contour(S_grid, T_grid, sigma, levels=levels,
                                   colors=color, linewidths=lw, linestyles='--')
                for level in levels:
                    if level in manual_labels:
                        inset.clabel(cs, fmt='%1.2f', fontsize=6, inline=False,
                                     manual=[manual_labels[level]])

            for a_idx, along in reversed(list(enumerate(along_vals))):
                T_col = temp.isel(along=a_idx).values
                S_col = sali.isel(along=a_idx).values
                mask = ~np.isnan(T_col) & ~np.isnan(S_col)
                if np.any(mask):
                    inset.scatter(S_col[mask], T_col[mask], s=0.5, color=cmap(norm(along)), marker='o')

            inset.set_xlim(*inset_xlim_local)
            inset.set_ylim(*inset_ylim_local)
            inset.set_xticks([])
            inset.set_yticks([])

            from matplotlib.patches import Rectangle
            box = Rectangle(
                (inset_xlim_local[0], inset_ylim_local[0]),
                inset_xlim_local[1] - inset_xlim_local[0],
                inset_ylim_local[1] - inset_ylim_local[0],
                edgecolor='black', facecolor='none', linestyle='--', linewidth=1)
            ax.add_patch(box)

    for j in range(n_panels, len(axes)):
        fig.delaxes(axes[j])

    fig.subplots_adjust(right=0.88, top=0.92)
    cbar_ax = fig.add_axes([0.9, 0.1, 0.02, 0.8])
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, cax=cbar_ax)
    cbar.set_label('Along-transect distance (m)', fontsize=12)
    fig.suptitle("T–S Diagrams Colored by Along Distance", fontsize=24)

    for ax in fig.axes:
        ax.set_rasterization_zorder(1)

plot_ts_shelf_basin_by_transect(
    cube,
    region_range=(0, 30000),
    target_years=[2024, 2025],
    target_months=list(range(1, 13)),
    xlim=(30.5, 34),
    ylim=(5, 16),
    ncols=4,
    inset_months=[1,2,3,4, 5, 6, 7, 8, 9,10,11,12],
    inset_limits_by_date={
        '2024-03-12': {'xlim': (32.5, 33), 'ylim': (7.5, 8.4)},
        '2024-04-16': {'xlim': (32.75, 33.5), 'ylim': (7.3, 8.5)},
        '2024-05-10': {'xlim': (32.7, 33.5), 'ylim': (7.2, 8.1)},
        '2024-05-16': {'xlim': (32.9, 33.6), 'ylim': (6.8, 7.9)},
        '2024-07-17': {'xlim': (33.3, 33.8), 'ylim': (6, 7.1)},
        '2024-07-23': {'xlim': (33.3, 33.8), 'ylim': (6, 7.1)},
        '2024-11-14': {'xlim': (33, 33.5), 'ylim': (6.6, 7.5)},
        '2024-11-19': {'xlim': (33, 33.5), 'ylim': (6.8, 8)},
        '2025-01-09': {'xlim': (32.5, 33.25), 'ylim': (6.8, 8.5)},
        '2025-01-14': {'xlim': (32.5, 33.25), 'ylim': (6.8, 8.5)},
        '2025-03-12': {'xlim': (32.5, 33.25), 'ylim': (7.3, 8.25)},
        '2025-04-30': {'xlim': (33, 33.8), 'ylim': (6.5, 7.5)},
    }
)